# Assigment 3

In [2]:
import igl
import numpy as np
import scipy as sp
from scipy import spatial
import meshplot as mp

from typing import Optional, Any

In [3]:
v, f = igl.read_triangle_mesh("data/cow.off")
mp.plot(v, f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# Vertex normal

In [4]:
# Standard face normal
def compute_standard_normals(vertices: list[Any], faces: list[Any]) -> list[Any]:
    """ Computes the unweighted average of surrounding face normals """
    return igl.per_vertex_normals(vertices, faces)

mp.plot(v, f, n=compute_standard_normals(v,f), shading={"flat": False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [5]:
# Area-weighted face normal
def compute_area_weighted_normals(vertices: list[Any], faces: list[Any]) -> list[Any]:
    """ Computes the area weighted average of surrounding face normals """
    return igl.per_vertex_normals(vertices, faces, weighting=igl.PER_VERTEX_NORMALS_WEIGHTING_TYPE_AREA )

mp.plot(v, f, n=compute_area_weighted_normals(v,f), shading={"flat": False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [6]:
# Mean-curvature normal
def compute_mean_curvature_normals(vertices: list[Any], faces: list[Any], eps: int) -> list[Any]:
    """ Uses thecotangent-weighted Laplacian to the mesh vertex positions to compute the normal weighted proportionally to mean curvature """
    
    # Step 1: Construct a Cotangent stiffness matrix (discrete laplacian)
    cotangent_matrix = igl.cotmatrix(vertices, faces)
   
    # Step 2: Construct the area matrix using VORONOI
    mass_matrix = igl.massmatrix(vertices, faces)
    
    minv = sp.sparse.diags(1 / mass_matrix.diagonal())
    hn = -minv.dot(cotangent_matrix.dot(vertices))
    h = np.linalg.norm(hn, axis=1)
    
    normals = np.array([ x / y for x, y in zip(hn, h)])
    normals[h < 1e-6,:] = compute_area_weighted_normals(vertices, faces)[h < 1e-6,:]
    return normals


    
n = compute_mean_curvature_normals(v, f, 10)
mp.plot(v, f, n=n, shading={"flat": False})


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [7]:
# PCA normal
def knn(vertex, tree, k=5):
    """ Computes the indices of the k nearest vertices to a vertex. """
    _, indices = tree.query(vertex, k=k)
    
    return indices


def compute_pca_normals(vertices: list[Any], faces: list[Any], k: int) -> list[Any]:
    """ Computes the PCA normal. """
    
    # Compute KDTree early     
    tree = spatial.KDTree(vertices)
    
    # Calculate PCA
    normals = list()
    for vertex in vertices:
        # Get neighbours         
        idx =  knn(vertex, tree, k=k)
        mv = np.mean(vertices[idx,:])
        centers = vertices[idx,:] - mv
        covariance = centers.T @ centers
        
        vals, vecs = np.linalg.eig(covariance)
        
        normal = np.min(vals)
        normals.append(normal)
    
    
    return np.array(normals)

n = compute_pca_normals(v, f, 5)
mp.plot(v, f, n=n, shading={"flat": False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [ ]:
# Quadratic fitting normal


# Curvature

In [8]:
# Gaussian curvature
def compute_gaussian_curvature(vertices:list[Any], faces:list[Any]) -> list[Any]:
    return igl.gaussian_curvature(vertices, faces)

mp.plot(v, f, compute_gaussian_curvature(v, f))


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [9]:
# Principal curvature
def compute_principal_curvature(vertices, faces):
    """ """
    v1, v2, k1, k2 = igl.principal_curvature(v, f)
    h = 0.5 * (k1 + k2)
    
    return v1, v2, h


v1, v2, h = compute_principal_curvature(v, f)
p = mp.plot(
    v, f, h, shading={"wireframe": False}, return_plot=True
)

avg = igl.avg_edge_length(v, f) / 2.0

p.add_lines(v + v1 * avg, v - v1 * avg, shading={"line_color": "red"})
p.add_lines(v + v2 * avg, v - v2 * avg, shading={"line_color": "green"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

2

# Smoothing with the Laplacian

In [12]:
from scipy.sparse.linalg import spsolve

In [51]:
# Explicit laplacian
def explicit_smoothing(vertices, faces, iter_count=1000, l=0.01):
    cotangents = igl.cotmatrix(vertices, faces)
    normals = igl.per_vertex_normals(vertices, faces)
    colors = np.linalg.norm(normals, axis=1)
    
    vs = [vertices]
    cs = [colors]
    
    for i in range(iter_count):
        mass_matrix = igl.massmatrix(vertices, faces, igl.MASSMATRIX_TYPE_VORONOI)
        minv = sp.sparse.diags(1 / mass_matrix.diagonal())
        L = minv.dot(cotangents)
        I = sp.sparse.identity(L.shape[0])
        s = (I + (1e-6) * L)
        
        vertices = s.dot(vertices)
        normals = igl.per_vertex_normals(vertices, faces)
        colors = np.linalg.norm(normals, axis=1)
        
        vs.append(vertices)
        cs.append(colors)

    return vs, cs

vs, cs = explicit_smoothing(v, f) 
plot = mp.subplot(vs[0], f, cs[0], s=[1, 3, 0])
mp.subplot(vs[500], f, cs[500], s=[1, 3, 1], data=plot)
mp.subplot(vs[999], f, cs[999], s=[1, 3, 2], data=plot)

In [29]:
# Implicit laplacian
def implicit_smoothing(vertices, faces, iter_count=1, l=20):
    cotangents = igl.cotmatrix(vertices, faces)
    normals = igl.per_vertex_normals(vertices, faces)
    colors = np.linalg.norm(normals, axis=1)
    
    vs = [vertices]
    cs = [colors]
    
    for i in range(iter_count):
        area_matrix = igl.massmatrix(vertices, faces, igl.MASSMATRIX_TYPE_BARYCENTRIC)
        s = area_matrix - (1 / l) * cotangents
        b = area_matrix.dot(vertices)
        vertices = spsolve(s, area_matrix.dot(vertices))
        normals = igl.per_vertex_normals(vertices, faces)
        colors = np.linalg.norm(normals, axis=1)
        vs.append(vertices)
        cs.append(colors)
    
    return vs, cs


vs, cs = implicit_smoothing(v, f) 
plot = mp.subplot(vs[0], f, cs[0], s=[1, 2, 0])
mp.subplot(vs[1], f, cs[1], s=[1, 2, 1], data=plot)